# Neural Network with Sklearn for Sentiment Analysis ( MLPClassifier )

In [41]:
# import library yang dibutuhkan

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import string
import re
from collections import Counter
from tqdm import tqdm
!pip install terbilang
from terbilang import Terbilang
# Import sklearn modul

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
# from sklearn.externals import joblib

from gensim.models import Word2Vec
import pickle

#import sastrawi
!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#tokenize
!pip install nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for terbilang: filename=terbilang-0.0.6-py3-none-any.whl size=4012 sha256=4e47931407b1bf95625443ecd7641e94ac84684095d9182afb9c3f363345fe1a
  Stored in directory: /root/.cache/pip/wheels/0c/d9/75/df95fb352dfd1bcb4135f3fa9aaded7a2f3ec993eb972e3253
Successfully built terbilang
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [139]:
# Preprosses Data

# stopwords_indonesia = list(set(stopwords.words('indonesian')))
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()
# Tambahan stopword
# add_stopword = ['ya', 'sembilan','belas', 'haha', 'ribu', 'puluh', 'dua', 'sih', 'nya' ]
# stopwords_indonesia.extend(add_stopword)



kamus_alay = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/new_kamusalay.csv', encoding='Latin-1',names=('alay','baku'))

# Melakukan Cleaning tweet tanpa menghapus stopwords
def text_proses(text):

    # Untuk stemmer kata
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    if type(text) == np.float64:
        return ""
    temp = re.sub(r'\b(USER|RT|URL)\b', '', text)
    temp = temp.lower()
    temp = re.sub(r"@[A-Za-z0-9_]+","", temp) #Menghilangkan Mention
    temp = re.sub("#[A-Za-z0-9_]+","", temp) #Menghilangkan Hashtag
    temp = re.sub(r'https://\S+', '', temp)
    # Melakukan subtitusi kata alay dengan kata baku
    temps = temp.split()
    for i in range(len(temps)):
        temp = temps[i]
        replacement = kamus_alay[kamus_alay['alay'] == temp]['baku']
        if not replacement.empty:
            temps[i] = replacement.iloc[0]
    temp = " ".join(temp.strip() for temp in temps)

    # Mengubah angka menjadi terbaca
    # temps = temp.split()
    # for i in range(len(temps)):
    #     temp = temps[i]
    #     if temps[i].isdigit() == True :
    #         temps[i] = Terbilang().parse(temps[i]).getresult()
    #     else :
    #         temps[i]
    # temp = " ".join(temps)

    # Melakukan stemming
    tokens = word_tokenize(temp)
    temp = ' '.join([word for word in tokens ])
    temp = stemmer.stem(temp)

    temp = re.sub(r'\\x..', '', temp) #Menghilangkan Emoticon
    temp = re.sub(r'&amp;', 'dan', temp)
    temp = re.sub(r'http\S+', '', temp) #Menghilangkan link
    temp = re.sub(r"www.\S+", "", temp) #Menghilangkan link
    temp = re.sub(r"\\[n|t]", "", temp) #Menghilangkan 'enter' dan 'tab'
    temp = re.sub("[^A-Za-z\s']","", temp) #Menghilangkan yang bukan huruf
    temp = temp.strip()

    return temp



In [28]:
# Upload Clean_data file

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/data_bersih.csv')
df.head(1)

,text,label,pro_text
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive,warung ini milik oleh usaha pabrik tahu yang s...


In [34]:
df.pro_text.duplicated().sum(), df.pro_text.isna().sum()  # Data sudah clean


(0, 0)

In [33]:
df = df.drop_duplicates(subset=['pro_text']).reset_index(drop=True)

## membuat vector untuk inputan ke Machine Learning

### 1. Dengan menggunakan Bag of Word ( CountVectorizer )

In [36]:
# Proses Feature extraction Bag of Word ( CountVectorizer )

data_preprocessed = df.pro_text.to_list()

count_vect = CountVectorizer(ngram_range=(1,2))

print('Feature extraction selesai')

Feature extraction selesai


### Best Model for Bag of words

In [37]:
# Cross Validation dengan df.pro_text & ngram_range=(1,2)
data_preprocessed = df.pro_text.to_list()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label


for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = count_vect.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = count_vect.transform(data_test)
    target_test = y[data[1]]

    clf = MLPClassifier(early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)




average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.82      0.85      0.84       655
     neutral       0.85      0.75      0.80       216
    positive       0.92      0.92      0.92      1311

    accuracy                           0.88      2182
   macro avg       0.86      0.84      0.85      2182
weighted avg       0.88      0.88      0.88      2182

Training ke- 2
              precision    recall  f1-score   support

    negative       0.85      0.86      0.86       671
     neutral       0.82      0.72      0.77       240
    positive       0.92      0.94      0.93      1271

    accuracy                           0.89      2182
   macro avg       0.87      0.84      0.85      2182
weighted avg       0.89      0.89      0.89      2182

Training ke- 3
              precision    recall  f1-score   support

    negative       0.86      0.84      0.85       696
     neutral       0.81      0.70      0.75       213
    positive       0.92      0

In [39]:
with open('model_BoW.pickle', 'wb') as f:
    pickle.dump(clf, f)

with open('bow_tokenizer.pickle', 'wb') as f:
    pickle.dump(count_vect, f)

In [68]:
# real world test
original_text = '''

Film Kartini bagus bangettt!!!! Wajib ditonton!

                '''
input_ = text_proses(original_text)
text = count_vect.transform([input_])
result = clf.predict(text)[0]

print('Sentence :', input_)
print('Sentiment :',result)


Sentence : film kartini bagus bangettt wajib tonton
Sentiment : positive


In [77]:
# Proses Feature extraction Bag of Word ( CountVectorizer )

count_vect2 = CountVectorizer(ngram_range=(1,1))

# Cross Validation dengan df.pro_text
data_preprocessed = df.pro_text.to_list()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label


for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = count_vect2.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = count_vect2.transform(data_test)
    target_test = y[data[1]]

    clf2 = MLPClassifier(hidden_layer_sizes=(100,100),early_stopping=True)
    clf2.fit(data_train, target_train)
    preds = clf2.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)




average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.80      0.86      0.83       655
     neutral       0.82      0.75      0.78       216
    positive       0.92      0.90      0.91      1311

    accuracy                           0.87      2182
   macro avg       0.85      0.84      0.84      2182
weighted avg       0.88      0.87      0.88      2182

Training ke- 2
              precision    recall  f1-score   support

    negative       0.84      0.85      0.85       671
     neutral       0.81      0.75      0.78       240
    positive       0.92      0.92      0.92      1271

    accuracy                           0.88      2182
   macro avg       0.86      0.84      0.85      2182
weighted avg       0.88      0.88      0.88      2182

Training ke- 3
              precision    recall  f1-score   support

    negative       0.83      0.86      0.85       696
     neutral       0.77      0.72      0.74       213
    positive       0.93      0

In [81]:
# real world test
original_text = '''


 Barusan bgt kelar nntn @filmziarah film lokal pertama yg gue tonton dan berhasil bikin nangis @bewe_bw speechless liat aktingnya @alitalit_
                '''
input_ = text_proses(original_text)
text = count_vect2.transform([input_])
result = clf2.predict(text)[0]

print('Sentence :', input_)
print('Sentiment :',result)


#Lapor mbah @CenayangFilm film ini keren. Worth to watch. Syukur2 bisa instrospeksi diri abis nonton.


Sentence : barusan banget selesai tonton film lokal pertama yang gue tonton dan hasil bikin menang speechless lihat akting
Sentiment : negative


### 2. Dengan menggunakan Tf-IDF ( TfidfVectorizer )

In [82]:
data_preprocessed = df.pro_text.to_list()

In [85]:
# Proses Feature extraction Tf-IDF ( TfidfVectorizer )
tfidf2 = TfidfVectorizer(ngram_range=(1,1))
# X_tfidf = tfidf.fit_transform(data_preprocessed)
print('Feature Extraction Selesai')

Feature Extraction Selesai


In [86]:
# Cross Validation dengan df.pro_text
data_preprocessed = df.pro_text.to_list()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label


for iteration, data in tqdm(enumerate(kf.split(np.array(data_preprocessed)), start=1)):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train = tfidf2.fit_transform(data_train)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test = tfidf2.transform(data_test)
    target_test = y[data[1]]

    clf3 = MLPClassifier(random_state=None, verbose=False, early_stopping=True)
    clf3.fit(data_train, target_train)
    preds = clf3.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))


    accuracies.append(accuracy)




average_accuracy = np.mean(accuracies)

# print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)
print('='*100)

1it [00:30, 30.63s/it]

Training ke- 1
              precision    recall  f1-score   support

    negative       0.79      0.85      0.82       655
     neutral       0.88      0.70      0.78       216
    positive       0.91      0.91      0.91      1311

    accuracy                           0.87      2182
   macro avg       0.86      0.82      0.84      2182
weighted avg       0.87      0.87      0.87      2182



2it [00:56, 27.57s/it]

Training ke- 2
              precision    recall  f1-score   support

    negative       0.85      0.84      0.84       671
     neutral       0.84      0.68      0.75       240
    positive       0.90      0.94      0.92      1271

    accuracy                           0.88      2182
   macro avg       0.86      0.82      0.84      2182
weighted avg       0.88      0.88      0.87      2182



3it [01:21, 26.56s/it]

Training ke- 3
              precision    recall  f1-score   support

    negative       0.82      0.83      0.82       696
     neutral       0.84      0.65      0.74       213
    positive       0.90      0.93      0.92      1273

    accuracy                           0.87      2182
   macro avg       0.86      0.80      0.83      2182
weighted avg       0.87      0.87      0.87      2182



4it [01:46, 25.88s/it]

Training ke- 4
              precision    recall  f1-score   support

    negative       0.81      0.84      0.83       707
     neutral       0.89      0.73      0.80       226
    positive       0.90      0.91      0.91      1248

    accuracy                           0.87      2181
   macro avg       0.87      0.83      0.85      2181
weighted avg       0.87      0.87      0.87      2181



5it [02:20, 28.14s/it]

Training ke- 5
              precision    recall  f1-score   support

    negative       0.81      0.84      0.83       672
     neutral       0.87      0.71      0.78       241
    positive       0.90      0.92      0.91      1268

    accuracy                           0.87      2181
   macro avg       0.86      0.82      0.84      2181
weighted avg       0.87      0.87      0.87      2181

rata-rata akurasi :  0.8719287186101449


In [73]:
with open('model_tfidf.pickle', 'wb') as f:
    pickle.dump(clf, f)

with open('tfidf_tokenizer.pickle', 'wb') as f:
    pickle.dump(tfidf, f)

In [89]:
# real world test
original_text = '''

	Lapor mbah @CenayangFilm film ini keren. Worth to watch. Syukur2 bisa instrospeksi diri abis nonton.

                '''
input_ = text_proses(original_text)
text = tfidf2.transform([input_])
result = clf3.predict(text)[0]

print('Sentence :', input_)
print('Sentiment :',result)


Sentence : lapor mbah film ini keren worth to watch syukur  bisa instrospeksi diri habis nonton
Sentiment : positive


### 3. Dengan menggunakan gensim model ( word2vec )

In [92]:
# Preprocess the text (tokenization)
nltk.download('punkt')
data_preprocessed = df.pro_text.to_list()
corpus = [nltk.word_tokenize(doc) for doc in data_preprocessed]

# Train Word2Vec model
model = Word2Vec(corpus, vector_size=400, window=5, min_count=1, workers=4)

# Generate document embeddings
X_w2v = [model.wv[doc].mean(axis=0) for doc in corpus]

print('Feature Extraction Selesai')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Feature Extraction Selesai


In [113]:
# Cross Validation

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label


for iteration, data in (enumerate(kf.split(X_w2v), start=1)):

    data_train = np.array(X_w2v)[data[0]]
    target_train = np.array(y)[data[0]]

    data_test = np.array(X_w2v)[data[1]]
    target_test = np.array(y)[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=(200,200) ,random_state=None, verbose=False, early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))

    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)

print('rata-rata akurasi : ', average_accuracy)
print('='*100)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.74      0.76      0.75       655
     neutral       0.73      0.68      0.70       216
    positive       0.89      0.89      0.89      1311

    accuracy                           0.83      2182
   macro avg       0.79      0.78      0.78      2182
weighted avg       0.83      0.83      0.83      2182

Training ke- 2
              precision    recall  f1-score   support

    negative       0.80      0.75      0.77       671
     neutral       0.71      0.72      0.72       240
    positive       0.89      0.92      0.90      1271

    accuracy                           0.84      2182
   macro avg       0.80      0.80      0.80      2182
weighted avg       0.84      0.84      0.84      2182

Training ke- 3
              precision    recall  f1-score   support

    negative       0.75      0.78      0.76       696
     neutral       0.75      0.59      0.66       213
    positive       0.89      0

In [115]:
# test dengan real world case

original_text = '''

	Lapor mbah @CenayangFilm film ini keren. Worth to watch. Syukur2 bisa instrospeksi diri abis nonton.


                '''
input_ = text_proses(original_text)
text = [nltk.word_tokenize(input_)]
model_w2v = Word2Vec(text, vector_size=400, window=5, min_count=1, workers=4)
data_w2v = [model_w2v.wv[doc].mean(axis=0) for doc in text]
result = clf.predict(data_w2v)[0]

print('Sentence : ',input_)
print('Sentiment : ',result)


Sentence :  lapor mbah film ini keren worth to watch syukur  bisa instrospeksi diri habis nonton
Sentiment :  negative


## 4. Kombinasi Bag of word & Tfidf

In [116]:
X_train, X_test, y_train, y_test = train_test_split(df.pro_text, df.label, test_size=0.25, random_state=46, stratify=df.label)



In [117]:
Vectorize = CountVectorizer(ngram_range=(1,1))
X_train_vec = Vectorize.fit_transform(X_train)
X_test_vec = Vectorize.transform(X_test)

X_train_vec

<8181x10249 sparse matrix of type '<class 'numpy.int64'>'
	with 202246 stored elements in Compressed Sparse Row format>

In [118]:
with open ('Vectorize_kombinasi.pickle','wb') as f:
  pickle.dump(Vectorize,f)

In [119]:
with open ('Vectorize_kombinasi.pickle', 'rb') as f:
  vectorx = pickle.load(f)

In [120]:
Transformer = TfidfTransformer()

X_train_tfidf = Transformer.fit_transform(X_train_vec)
X_test_tfidf = Transformer.transform(X_test_vec)

X_train_tfidf

<8181x10249 sparse matrix of type '<class 'numpy.float64'>'
	with 202246 stored elements in Compressed Sparse Row format>

In [121]:
with open ('Transformer_kombinasi.pickle','wb') as f:
  pickle.dump(Transformer,f)

In [122]:

import time

start_ = time.time()
clf = MLPClassifier(hidden_layer_sizes=(128,128,128),early_stopping=True)
clf.fit(X_train_tfidf, y_train)

y_pred = clf.predict(X_test_tfidf)
end_ = time.time()
print('Waktu training - predict :', end_ - start_)
print(classification_report(y_test, y_pred))

Waktu training - predict : 26.684258222579956
              precision    recall  f1-score   support

    negative       0.84      0.79      0.81       850
     neutral       0.81      0.71      0.76       284
    positive       0.89      0.94      0.91      1593

    accuracy                           0.87      2727
   macro avg       0.85      0.81      0.83      2727
weighted avg       0.86      0.87      0.86      2727



In [ ]:
# dir(clf)
# clf.__sizeof__()

In [126]:
# Cross Validation dengan df.pro_text
data_preprocessed = df.pro_text.to_list()

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = df.label

for iteration, data in enumerate(kf.split(np.array(data_preprocessed)), start=1):

    data_train = np.array(data_preprocessed)[data[0]]
    data_train_vec = Vectorize.fit_transform(data_train)
    data_train_tfidf = Transformer.fit_transform(data_train_vec)
    target_train = y[data[0]]

    data_test = np.array(data_preprocessed)[data[1]]
    data_test_vec = Vectorize.transform(data_test)
    data_test_tfidf = Transformer.transform(data_test_vec)
    target_test = y[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=(200,200,200),verbose=False, early_stopping=True)
    clf.fit(data_train_tfidf, target_train)
    preds = clf.predict(data_test_tfidf)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))

    accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)

print('rata-rata akurasi : ', average_accuracy)
print('='*100)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.78      0.87      0.82       655
     neutral       0.83      0.73      0.78       216
    positive       0.93      0.89      0.91      1311

    accuracy                           0.87      2182
   macro avg       0.85      0.83      0.84      2182
weighted avg       0.87      0.87      0.87      2182

Training ke- 2
              precision    recall  f1-score   support

    negative       0.86      0.80      0.83       671
     neutral       0.79      0.62      0.70       240
    positive       0.88      0.95      0.91      1271

    accuracy                           0.87      2182
   macro avg       0.84      0.79      0.81      2182
weighted avg       0.87      0.87      0.86      2182

Training ke- 3
              precision    recall  f1-score   support

    negative       0.82      0.81      0.82       696
     neutral       0.82      0.68      0.74       213
    positive       0.90      0

In [128]:
with open('model_mlp_kombinasi.pickle', 'wb') as f:
    pickle.dump(clf, f)



In [148]:
# real world test
original_text = '''

Postingan 28 Januari 2017, meneduhkan. Tetep #AHY

                '''
input_ = text_proses(original_text)
text = Vectorize.transform([input_])
text = Transformer.transform(text)
result = clf.predict(text)[0]

print('Sentence : ',input_)
print('Sentiment : ',result)


Sentence :  postingan dua puluh delapan januari  teduh tetap
Sentiment :  neutral
